In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Importing Libraries
!pip3 install graphviz
!pip3 install dask
!pip install "dask[complete]" 
!pip3 install toolz
!pip3 install cloudpickle
!pip install scikit-learn -U
# https://www.youtube.com/watch?v=ieW3G7ZzRZ0
# https://github.com/dask/dask-tutorial
import dask.dataframe as dd#similar to pandas

import pandas as pd#pandas to create small dataframes 

# if this doesnt work refere install_folium.JPG in drive
import folium #open street map

# unix time: https://www.unixtimestamp.com/
import datetime #Convert to unix time

import time #Convert to unix time

# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays

# matplotlib: used to plot graphs
import matplotlib
# matplotlib.use('nbagg') : matplotlib uses this protocall which makes plots more user intractive like zoom in and zoom out
matplotlib.use('nbagg')
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  

# this lib is used while we calculate the stight line distance between two (lat,lon) pairs in miles
!pip install gpxpy
import gpxpy.geo #Get the haversine distance

from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os

# download migwin: https://mingw-w64.org/doku.php/download/mingw-builds
# install it in your system and keep the path, migw_path ='installed path'
mingw_path = 'C:\Program Files (x86)\mingw-w64\i686-8.1.0-posix-dwarf-rt_v6-rev0\mingw32\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

# to install xgboost: pip3 install xgboost
# if it didnt happen check install_xgboost.JPG
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

import datetime
import time
from IPython.display import display


In [ ]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Clustering

In [ ]:
def unix2datetime(unx):
  return datetime.datetime.utcfromtimestamp(int(unx))

def datetime2unix(dt):
  return time.mktime(dt.timetuple())

def make_clusters(df, n_clusters):
  coords = df[['pickup_latitude', 'pickup_longitude']].values
  kmns = MiniBatchKMeans(n_clusters=n_clusters, batch_size=10000,random_state=0).fit(coords)
  df['pickup_cluster'] = kmns.predict(coords)
  return kmns


def make_bins(df, bin_mins):
  min_unix = df['pickup_times'].min()
  min_dt = unix2datetime(min_unix)
  year, month = min_dt.year, min_dt.month
  start_dt = datetime.date(year, month, 1)
  start_unix = datetime2unix(start_dt)

  period = bin_mins * 60
  bins = []
  for pu in df['pickup_times'].values:
    bins.append(int((pu - start_unix) // period))

  df['pickup_bins'] = bins

In [ ]:
def visualize_cluster_centers_map(centers):
  cluster_len = len(cluster_centers)
  map_osm = folium.Map(location=[40.734695, -73.990372], tiles='Stamen Toner')
  for i in range(cluster_len):
      folium.Marker(list((cluster_centers[i][0],cluster_centers[i][1])), popup=(str(cluster_centers[i][0])+str(cluster_centers[i][1]))).add_to(map_osm)
  display(map_osm)

def plot_clusters(df):
  city_long_border = (-74.03, -73.75)
  city_lat_border = (40.63, 40.85)
  fig, ax = plt.subplots(figsize=(15,15),ncols=1, nrows=1)
  ax.scatter(df.pickup_longitude.values[:100000], df.pickup_latitude.values[:100000], s=10, lw=0,
                c=df.pickup_cluster.values[:100000], cmap='tab20', alpha=0.2)
  ax.set_xlim(city_long_border)
  ax.set_ylim(city_lat_border)
  ax.set_xlabel('Longitude')
  ax.set_ylabel('Latitude')
  plt.show()

In [ ]:
def df2numpy(df, num_clusters):
  num_bins = df['pickup_bins'].max() + 1
  data = np.zeros((num_bins,num_clusters))
  for i in range(num_bins):
    cur_bin = df[df['pickup_bins'] == i]['pickup_cluster']
    for pu in cur_bin:
      data[i][pu] += 1
  return data

def get_name(year, month, num_clusters, bin_mins):
  if month <= 10:
    month = "0" + str(month)
  name = "{}-{}_{}_{}.npy".format(year, month, num_clusters, bin_mins)
  return name

def save_data(data, year, month, num_clusters, bin_mins, save_dir="/content/gdrive/MyDrive/New_York_Data/Clustered/"):
  name = get_name(year, month, num_clusters, bin_mins)
  save_path = os.path.join(save_dir, name)
  if os.path.exists(save_path):
    print("{} already exists, skipping...".format(save_path))
    return save_path
  with open(save_path, "wb") as f:
    np.save(f, data)
  return save_path

In [ ]:
def get_cluster_name(year, month, num_clusters, suffix=''):
  if month <= 10:
    month = "0" + str(month)
  name = "cluster_centers_{}-{}{}_{}.npy".format(year, month, suffix, num_clusters)
  return name

def save_cluster_centers(data, year, month, num_clusters, \
                         save_dir="/content/gdrive/MyDrive/New_York_Data/Clustered/"):
  name = get_cluster_name(year, month, num_clusters)
  path = os.path.join(save_dir, name)
  if os.path.exists(path):
    print("{} already exists, skipping...".format(path))
  else:
    with open(path, 'wb') as f:
      np.save(f, data, allow_pickle=True)
  return path


def load_cleaned_data(year, month, load_dir="/content/gdrive/MyDrive/New_York_Data/clean/"):
  if month <= 10:
    month = "0" + str(month)
  name = "clean_yellow_tripdata_{}-{}.csv".format(year, month)
  path = os.path.join(load_dir, name)
  if not os.path.exists(path):
    raise Exception("{} does not exist...".format(path))
  cleaned_data = pd.read_csv(path)
  return cleaned_data

def preprocess2npy(df, year, month, num_clusters, bin_mins, save_dir="/content/gdrive/MyDrive/New_York_Data/Clustered/"):
  print("Making clusters...")
  kmns = make_clusters(df, num_clusters)
  print("Saving cluster centers...")
  path = save_cluster_centers(kmns.cluster_centers_, year, month, num_clusters, save_dir)
  print("Cluster centers saved to {} ...".format(path))
  print("Making bins...")
  make_bins(df, bin_mins)
  data = df2numpy(df, num_clusters)
  print("Saving...")
  save_path = save_data(data, year, month, num_clusters, bin_mins, save_dir)
  print("Saved to", save_path)
  return save_path

In [ ]:
cleaned_data = pd.read_csv("/content/gdrive/MyDrive/New_York_Data/clean/clean_yellow_tripdata_2016-02.csv")

In [ ]:
preprocess2npy(cleaned_data, 2016, 2, 125, 30)

Making clusters...
Saving cluster centers...
/content/gdrive/MyDrive/New_York_Data/Clustered/cluster_centers_2016-02_125.npy already exists, skipping...
Cluster centers saved to /content/gdrive/MyDrive/New_York_Data/Clustered/cluster_centers_2016-02_125.npy ...
Making bins...
Saving...
/content/gdrive/MyDrive/New_York_Data/Clustered/2016-02_125_30.npy already exists, skipping...
Saved to /content/gdrive/MyDrive/New_York_Data/Clustered/2016-02_125_30.npy


'/content/gdrive/MyDrive/New_York_Data/Clustered/2016-02_125_30.npy'

# Preprocess data

In [ ]:
def load_data(year, month, num_clusters, bin_mins, \
              load_dir="/content/gdrive/MyDrive/New_York_Data/Clustered"):
  name = get_name(year, month, num_clusters, bin_mins)
  path = os.path.join(load_dir, name)
  if not os.path.exists(path):
    raise Exception("Could not find {}".format(path))
  
  with open(path, 'rb') as f:
    data = np.load(f, allow_pickle=True)
  return data

In [ ]:
def get_weekdays(year, month, data, bin_mins):
  start_dt = datetime.date(year, month, 1)
  start_wd = start_dt.weekday()
  day_sec = 60 * 60 * 24
  week_sec = day_sec * 7
  weekdays = []
  for i in range(len(data)):
    secs = i * 60 * bin_mins
    weekday = ((secs % week_sec) // day_sec + start_wd) % 7
    weekdays.append(weekday)
  return weekdays

def load_cluster_centers(year, month, num_clusters, suffix="", \
            load_dir="/content/gdrive/MyDrive/New_York_Data/Clustered",\
      ):
  name = get_cluster_name(year, month, num_clusters)
  path = os.path.join(load_dir, name)
  if not os.path.exists(path):
    raise Exception("Could not find {}".format(path))

  with open(path, 'rb') as f:
    data = np.load(f, allow_pickle=True)
  return data

In [ ]:
def make_seq(data, cluster_centers, weekdays, daybins, window_size=10):
  seq_data = []
  num_bins = len(data)
  num_clusters = len(data[0])
  lat = cluster_centers[:, 1]
  lon = cluster_centers[:, 0]
  for i in range(num_clusters):
    for j in range(num_bins - window_size):
      seq = list(data[j:j+window_size, i])
      for item in [lat[i], lon[i], weekdays[j], daybins[j], data[j+window_size][i]]:
        seq.append(item)
      seq_data.append(seq)

  return np.array(seq_data)

def train_test_split(data, weekdays, daybins, train_ratio=0.7):
  train_size = int(len(data) * train_ratio)
  train_data = data[:train_size]
  test_data = data[train_size:]
  train_weekdays = weekdays[:train_size]
  test_weekdays = weekdays[train_size:]
  train_daybins = daybins[:train_size]
  test_daybins = daybins[train_size:]
  return train_data, test_data, train_weekdays, test_weekdays, train_daybins, test_daybins

In [ ]:
def get_full_data(year, month, num_clusters, bin_mins, \
                  load_dir="/content/gdrive/MyDrive/New_York_Data/Clustered"):
  data = load_data(year, month, num_clusters, bin_mins)
  print(data.shape)
  cluster_centers = load_cluster_centers(year, month, num_clusters)
  weekdays = get_weekdays(year, month, data, bin_mins)
  daybins = np.mod(np.arange(len(data)),24*60//bin_mins)
  train_data, test_data, train_weekdays, test_weekdays, train_daybins, test_daybins = train_test_split(data, weekdays, daybins)
  train_data = make_seq(train_data, cluster_centers, train_weekdays, train_daybins)
  test_data = make_seq(test_data, cluster_centers, test_weekdays, test_daybins)
  x_train, y_train = train_data[:, :-1], train_data[:, -1]
  x_test, y_test = test_data[:, :-1], test_data[:, -1]

  return x_train, y_train, x_test, y_test

In [ ]:
def get_weekly_periodic_data(year, month, day, num_clusters, bin_mins, \
                  load_dir="/content/gdrive/MyDrive/New_York_Data/Clustered"):
  data = load_data(year, month, num_clusters, bin_mins)
  cluster_centers = load_cluster_centers(year, month, num_clusters)
  weekdays = get_weekdays(year, month, data, bin_mins)
  daybins = np.mod(np.arange(len(data)),24*60//bin_mins)
  daybins_ext = np.repeat(daybins[:,np.newaxis], len(cluster_centers), axis=1)
  lat = np.repeat(cluster_centers[np.newaxis, :, 1], len(data), axis=0)
  lon = np.repeat(cluster_centers[np.newaxis, :, 0], len(data), axis=0)

  data = np.stack((daybins_ext, lat, lon, data), axis=2)
  train_data, test_data, train_weekdays, test_weekdays, train_daybins, test_daybins = train_test_split(data, weekdays, daybins)

  train_data = train_data[np.array(train_weekdays)==day].reshape(-1,4)
  test_data = test_data[np.array(test_weekdays)==day].reshape(-1,4)
  
  x_train, y_train = train_data[:, :-1], train_data[:, -1]
  x_test, y_test = test_data[:, :-1], test_data[:, -1]

  return x_train, y_train, x_test, y_test

#First Decision Tree with Periodic Boundary Conditions



In [ ]:
year = 2016
month = 2
bin_mins = 30
num_clusters = 125

In [ ]:
cleaned_data = load_cleaned_data(year, month)
preprocess2npy(cleaned_data, year, month, num_clusters, bin_mins)

Making clusters...
Saving cluster centers...
/content/gdrive/MyDrive/New_York_Data/Clustered/cluster_centers_2016-02_125.npy already exists, skipping...
Cluster centers saved to /content/gdrive/MyDrive/New_York_Data/Clustered/cluster_centers_2016-02_125.npy ...
Making bins...
Saving...
/content/gdrive/MyDrive/New_York_Data/Clustered/2016-02_125_30.npy already exists, skipping...
Saved to /content/gdrive/MyDrive/New_York_Data/Clustered/2016-02_125_30.npy


'/content/gdrive/MyDrive/New_York_Data/Clustered/2016-02_125_30.npy'

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

def mape(y_true, y_pred):
  err = mean_absolute_error(y_true, y_pred) / (sum(y_true) / len(y_true))
  return err

weekday_regressors = []
for day in range(7):
  x_train, y_train, x_test, y_test = get_weekly_periodic_data(year, month, day, num_clusters, bin_mins)
  weekday_regressors.append(RandomForestRegressor(max_features='sqrt',min_samples_leaf=9,min_samples_split=7,n_estimators=79, n_jobs=-1))
  weekday_regressors[day].fit(x_train, y_train)

print("Reassuringly, the Sunday classifier performs significantly better on the weekend than on workdays:")
for i in weekday_regressors:
  pred = i.predict(x_test)
  print(mape(pred, y_test))

Reassuringly, the Sunday classifier performs significantly better on the weekend than on workdays:
0.4229569494195547
0.496461347141585
0.4883327588022728
0.486269060603158
0.44653599485439893
0.3135395272612615
0.2348956080376671


# Second Decision Tree for Time-Series Forecast

In [ ]:
year = 2016
month = 2
bin_mins = 30
num_clusters = 125

In [ ]:
cleaned_data = load_cleaned_data(year, month)
preprocess2npy(cleaned_data, year, month, num_clusters, bin_mins)

Making clusters...
Saving cluster centers...
/content/gdrive/MyDrive/New_York_Data/Clustered/cluster_centers_2016-02_125.npy already exists, skipping...
Cluster centers saved to /content/gdrive/MyDrive/New_York_Data/Clustered/cluster_centers_2016-02_125.npy ...
Making bins...
Saving...
/content/gdrive/MyDrive/New_York_Data/Clustered/2016-02_125_30.npy already exists, skipping...
Saved to /content/gdrive/MyDrive/New_York_Data/Clustered/2016-02_125_30.npy


'/content/gdrive/MyDrive/New_York_Data/Clustered/2016-02_125_30.npy'

In [ ]:
x_train, y_train, x_test, y_test = get_full_data(year, month, num_clusters, bin_mins)

(1392, 125)


In [ ]:
print(np.abs(y_train).mean())

predictions = []
for r in weekday_regressors:
  predictions.append(r.predict( x_train[:, [-1,-4,-3]]) )

for i in range(len(x_train)):
  weekday = int(x_train[i][-2])
  y_train[i] -= predictions[weekday][i]

print(np.abs(y_train).mean())

64.11177593360996
42.12251445116694


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [ ]:
def mape(y_true, y_pred):
  err = mean_absolute_error(y_true, y_pred) / (sum(y_true) / len(y_true))
  return err

In [ ]:
from sklearn.ensemble import RandomForestClassifier

regr1 = RandomForestRegressor(max_features='sqrt',min_samples_leaf=9,min_samples_split=7,n_estimators=100, n_jobs=-1)
regr1.fit(x_train, y_train)

predictions = []
for r in weekday_regressors:
  predictions.append(r.predict( x_test[:, [-1,-4,-3]]) )

y_pred = np.zeros(len(x_test))
reg_pred = regr1.predict(x_test)

for i in range(len(x_test)):
  weekday = int(x_test[i][-2])
  y_pred[i] = predictions[weekday][i] + reg_pred[i]

In [ ]:
mape(y_test, y_pred)

0.20686466532085795